In [1]:
##############################################################################################################################
##############################################################################################################################
##PYTHON CODE (notebook, .ipynb file using https://jupyter.org/ for https://www.python.org/) FOR A PERIOD LIFE TABLE,
##     REPRODUCED FROM ONLINE R MATERIALS FOR A 2006 FORMAL DEMOGRAPHY WORKSHOP AT STANFORD UNIVERSITY
##     R CODE LINKED TO AT: https://applieddemogtoolbox.github.io/#StanfordCourseLifeTable
##
##OCTOBER 2022
##edyhsgr@gmail.com
##
##IF YOU WOULD LIKE TO USE, SHARE OR REPRODUCE THIS CODE, PLEASE CITE THE SOURCE
##
##THERE IS NO WARRANTY FOR THIS CODE
##
##USEFUL PYTHON INFO BY UC DAVIS PROF NORM MATLOFF: https://web.cs.ucdavis.edu/~matloff/matloff/public_html/python.html 
##AND YOUTUBER PROGRAMMING WITH MOSH: https://www.youtube.com/watch?v=kqtD5dpn9C8&list=PLTjRvDozrdlxj5wgH4qkvwSOdHLOCx10f&index=18
##############################################################################################################################

##Calling libraries aka packages
import pandas as pandas  #typically 'import pandas as pd'; a Python library with data analysis tools including to plot, can install eg with 'pip install pandas'
pandas.options.mode.chained_assignment = None  # default='warn'  #To avoid pandas 'SettingWithCopyWarning'
import numpy as numpy  #typically 'import numpy as np'; a Python library with numerical analysis tools, can install eg with 'pip install numpy'

##############################################################################################################################
#STEP 1: Read in and review the population and death data
##############################################################################################################################

females = pandas.read_csv("https://github.com/AppliedDemogToolbox/StanfordCourseLifeTable/raw/master/StanfordCourseMortalityData.csv")
females


,Gender,Age.Description,Death.Count,Population,Crude.Death.Rate
0,Female,Under 1 Year,49016,7846166,624.7
1,Female,1- 4 years,8689,30110500,28.9
2,Female,5- 9 years,5595,39639456,14.1
3,Female,10-14 years,6399,40360290,15.9
4,Female,15-19 years,15621,39339459,39.7
5,Female,20-24 years,17776,37857523,47.0
6,Female,25-34 years,50896,79035547,64.4
7,Female,35-44 years,131696,90536189,145.5
8,Female,45-54 years,245138,78172750,313.6
9,Female,55-64 years,394940,51980118,759.8


In [2]:
##############################################################################################################################
#STEP 2: Read in or create the fundamental pieces of the life table (age groupings, deaths by age, population by age ->death rates by age
##############################################################################################################################

x = [0,1,5,10,15,20,25,35,45,55,65,75,85]
nDx = females['Death.Count']   
nKx = females['Population']
nMx = nDx / nKx


In [3]:
##############################################################################################################################
#STEP 3: Read in the period life table function
##############################################################################################################################

##Function making and using
def life_table(x,nMx):  
    ## simple lifetable using Keyfitz and Flieger separation factors and 
    ## exponential tail of death distribution (to close out life table)
    b0 = 0.07   
    b1 = 1.7      
    nmax = len(x)
    #nMx = nDx/nKx   
    n = numpy.diff(x)
    n = numpy.append(n,999)
    nax = n / 2                            # default to .5 of interval
    nax[0] = b0 + b1 * nMx[0]              # from Keyfitz & Flieger(1968)
    if n[1] == 4: nax[1] = 1.5                   #EddieH note: modified to if statement to include complete (single-year) tables
    nax[nmax-1] = 1/nMx[nmax-1]            # e_x at open age interval
    nqx = (n*nMx) / (1 + (n-nax)*nMx)
    for i in range(len(nqx)):              # necessary for high nMx
        if nqx[i] > 1: nqx[i] = 1
    nqx[nmax-1] = 1.0    
    lx = numpy.cumprod(1-nqx)              # survivorship lx
    lx = lx[:len(lx)-1]
    lx = numpy.append(1,lx)
    ndx = lx * nqx 
    #EddieH note: Below calculation modified from original nLx = n*lx - nax*ndx     # equivalent to n*l(x+n) + (n-nax)*ndx
    nLx = n*lx - (n-nax)*ndx               # equivalent to n*l(x+n) + nax*ndx
    nLx[nmax-1] = lx[nmax-1]*nax[nmax-1]
    Tx = numpy.flipud(numpy.flipud(nLx).cumsum())
    ex = Tx/lx
    lt = pandas.DataFrame()
    lt['x'] = x
    lt['nax'] = nax
    lt['nMx'] = nMx
    lt['nqx'] = nqx
    lt['lx'] = lx
    lt['ndx'] = ndx
    lt['nLx'] = nLx
    lt['Tx'] = Tx
    lt['ex'] = ex
    return lt


In [4]:
##############################################################################################################################
#STEP 4: Apply the function to the data, and review the created life table
##############################################################################################################################

females_life_table = life_table(x,nMx)
females_life_table

#females_life_table.to_csv("###.csv")


,x,nax,nMx,nqx,lx,ndx,nLx,Tx,ex
0,0,0.080620,0.006247,0.006211,1.000000,0.006211,0.994289,79.623301,79.623301
1,1,1.500000,0.000289,0.001153,0.993789,0.001146,3.972288,78.629011,79.120464
2,5,2.500000,0.000141,0.000705,0.992642,0.000700,4.961461,74.656723,75.210099
3,10,2.500000,0.000159,0.000792,0.991942,0.000786,4.957745,69.695262,70.261431
4,15,2.500000,0.000397,0.001983,0.991156,0.001966,4.950865,64.737517,65.315169
5,20,2.500000,0.000470,0.002345,0.989190,0.002320,4.940151,59.786653,60.440007
6,25,5.000000,0.000644,0.006419,0.986870,0.006335,9.837030,54.846502,55.576196
7,35,5.000000,0.001455,0.014441,0.980536,0.014160,9.734556,45.009471,45.902940
8,45,5.000000,0.003136,0.030874,0.966376,0.029836,9.514574,35.274915,36.502282
9,55,5.000000,0.007598,0.073198,0.936539,0.068553,9.022628,25.760340,27.505882
